In [1]:
big_file = open('/Users/liamhettinger/Documents/Fall_MSBA/Applied_Data_Analytics/Homework/word-files/big.txt').read()

In [7]:
import sqlite3
import nltk
#import random
#import numpy as np
#from collections import Counter 
from collections import defaultdict
from nltk import FreqDist
from nltk.corpus import stopwords
import nltk
from nltk import FreqDist
from string import punctuation
import string

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [4]:
punctuation = set(punctuation)
punctuation.add("’")

In [5]:
query_results = convention_cur.execute(
                            '''
                                SELECT text, party
                                FROM conventions
                                WHERE speaker != "Unknown"
                            ''')

In [8]:
convention_data = defaultdict(str)

for row in query_results :
    text, party = row

    text = "".join([ch for ch in text if ch not in punctuation])    
    text = [w.lower() for w in text.split() if w.isalpha()]
    
    convention_data[party] += " ".join(text) + " "

In [18]:
def get_patterns(text, num_words=10):
    
    text = text.split()
    text = [word for word in text if not isinstance(word, int)]
    text = [word.lower() for word in text]
    text = [word for word in text if word not in string.punctuation]
    
    #Not sure if we wanted to remove stop words but if intended this code could do it
    #sw = stopwords.words('english')
    #text = [word for word in text if word not in sw] 
    
    # Calculate total tokens
    total_tokens = len(text)
    
    # Calculating unique tokens
    unique_tokens = len(list(FreqDist(text).keys()))

    #Calculating average token length
    avg_token_len = sum([len(word) for word in text]) / len(text)
    
    #Calculating lexical diversity
    lex_diversity = unique_tokens/total_tokens
    
    #Calculating top 10 
    top_words = FreqDist(text).most_common(num_words)
        
    
    #Results to a dictionary
    results = {'tokens':total_tokens,
               'unique_tokens':unique_tokens,
               'avg_token_length':avg_token_len,
               'lexical_diversity':lex_diversity,
               'top_words':top_words}
    
    return(results)

In [19]:
def group_comparison(corpus_1, corpus_2, num_words = 10, ratio_cutoff = 5):
    c1_results = get_patterns(corpus_1)
    c2_results = get_patterns(corpus_2)
    
    #Corpus 1 cleaning
    corpus_1 = corpus_1.split()
    corpus_1 = [word for word in corpus_1 if not isinstance(word, int)]
    corpus_1 = [word.lower() for word in corpus_1]
    corpus_1 = [word for word in corpus_1 if word not in string.punctuation]


    #Corpus 2 cleaning
    corpus_2 = corpus_2.split()
    corpus_2 = [word for word in corpus_2 if not isinstance(word, int)]
    corpus_2 = [word.lower() for word in corpus_2]
    corpus_2 = [word for word in corpus_2 if word not in string.punctuation]


    #ratio cutoff = 5
    #ratio cutoff is the number of words in each dataset.
    c1_dict = {}
    for key, value in FreqDist(corpus_1).items():
        if value > ratio_cutoff:
            c1_dict[key] = value/len(corpus_1)
        else:
            continue

    c2_dict = {}
    for key, value in FreqDist(corpus_2).items():
        if value > ratio_cutoff:
            c2_dict[key] = value/len(corpus_2)
        else:
            continue

    c_1VSc_2 = {}
    c_2VSc_1 = {}
    
    #adding comparison to dictionaries
    for key in c1_dict.keys():
        if key in c2_dict.keys():
            c_1VSc_2[key] = c1_dict[key]/c2_dict[key]
            c_2VSc_1[key] = c2_dict[key]/c1_dict[key]
    
    #sorting dictionary
    #num words is the number of words to be displayed
    one_vs_two = sorted(c_1VSc_2.items(), key=lambda k: k[1], reverse = True)[:num_words]
    two_vs_one = sorted(c_2VSc_1.items(), key=lambda k: k[1], reverse = True)[:num_words]
    
    #Adding to dictionary
    comparison_dict = {'one': c1_results,
                      'two': c2_results,
                      'one_vs_two': one_vs_two,
                      'two_vs_one': two_vs_one}
    
    return(comparison_dict)

In [20]:
group_comparison(convention_data['Republican'], convention_data['Democratic'], num_words = 10, ratio_cutoff = 5)

{'one': {'tokens': 81113,
  'unique_tokens': 7014,
  'avg_token_length': 4.589165731756932,
  'lexical_diversity': 0.08647195887219064,
  'top_words': [('the', 3817),
   ('and', 2923),
   ('to', 2566),
   ('of', 1975),
   ('a', 1691),
   ('in', 1503),
   ('i', 1173),
   ('that', 1103),
   ('for', 1036),
   ('we', 1033)]},
 'two': {'tokens': 71970,
  'unique_tokens': 6001,
  'avg_token_length': 4.41879949979158,
  'lexical_diversity': 0.08338196470751702,
  'top_words': [('the', 3067),
   ('and', 2729),
   ('to', 2490),
   ('of', 1771),
   ('a', 1608),
   ('we', 1190),
   ('in', 1153),
   ('our', 1075),
   ('that', 1066),
   ('for', 1043)]},
 'one_vs_two': [('freedom', 6.78135969044948),
  ('heroes', 6.122236879415136),
  ('trade', 5.767324596550492),
  ('greatest', 5.577193016444432),
  ('bless', 5.323684242969685),
  ('cities', 5.3236842429696845),
  ('police', 5.260307049600998),
  ('land', 5.162360478031209),
  ('grateful', 4.05614037559595),
  ('democrat', 3.9524322409926445)],
 't

In [21]:
group_comparison(big_file, convention_data['Democratic'], num_words = 10, ratio_cutoff = 5)

{'one': {'tokens': 1095046,
  'unique_tokens': 75151,
  'avg_token_length': 4.861016797467869,
  'lexical_diversity': 0.06862816721854607,
  'top_words': [('the', 78172),
   ('of', 39450),
   ('and', 37009),
   ('to', 28295),
   ('in', 21393),
   ('a', 20610),
   ('that', 11483),
   ('he', 11481),
   ('was', 11159),
   ('his', 9961)]},
 'two': {'tokens': 71970,
  'unique_tokens': 6001,
  'avg_token_length': 4.41879949979158,
  'lexical_diversity': 0.08338196470751702,
  'top_words': [('the', 3067),
   ('and', 2729),
   ('to', 2490),
   ('of', 1771),
   ('a', 1608),
   ('we', 1190),
   ('in', 1153),
   ('our', 1075),
   ('that', 1066),
   ('for', 1043)]},
 'one_vs_two': [('such', 12.975650598891997),
  ('which', 8.403449870294642),
  ('andrew', 6.788274517365351),
  ('looked', 5.476938868321513),
  ('began', 5.4185181870594175),
  ('may', 5.085337739236525),
  ('though', 4.432669190761545),
  ('soon', 4.392504972393854),
  ('between', 4.199716724228936),
  ('eyes', 4.140565784451065)],
